<p style="text-align:center">
    <a href="https://skills.network/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2022-01-01" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


# **Hands-on Lab: Interactive Visual Analytics with Folium**


Estimated time needed: **40** minutes


The launch success rate may depend on many factors such as payload mass, orbit type, and so on. It may also depend on the location and proximities of a launch site, i.e., the initial position of rocket trajectories. Finding an optimal location for building a launch site certainly involves many factors and hopefully we could discover some of the factors by analyzing the existing launch site locations.


In the previous exploratory data analysis labs, you have visualized the SpaceX launch dataset using `matplotlib` and `seaborn` and discovered some preliminary correlations between the launch site and success rates. In this lab, you will be performing more interactive visual analytics using `Folium`.


## Objectives


This lab contains the following tasks:

*   **TASK 1:** Mark all launch sites on a map
*   **TASK 2:** Mark the success/failed launches for each site on the map
*   **TASK 3:** Calculate the distances between a launch site to its proximities

After completed the above tasks, you should be able to find some geographical patterns about launch sites.


Let's first import required Python packages for this lab:


In [4]:
import piplite
await piplite.install(['folium'])
await piplite.install(['pandas'])

In [5]:
import folium
import pandas as pd

In [6]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

If you need to refresh your memory about folium, you may download and refer to this previous folium lab:


[Generating Maps with Python](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DV0101EN-SkillsNetwork/labs/v4/DV0101EN-Exercise-Generating-Maps-in-Python.ipynb)


In [7]:
import folium

# Criar um mapa com um ponto de início
site_map = folium.Map(location=[28.573255, -80.646895], zoom_start=5)

# Lista com os locais de lançamento e suas coordenadas
launch_sites = [
    {'name': 'CCAFS LC-40', 'latitude': 28.562302, 'longitude': -80.577356},
    {'name': 'VAFB SLC-4E', 'latitude': 34.632093, 'longitude': -120.610829},
    {'name': 'KSC LC-39A', 'latitude': 28.573255, 'longitude': -80.646895},
    {'name': 'CCAFS SLC-40', 'latitude': 28.563197, 'longitude': -80.576820}
]

# Adicionar marcadores para cada local de lançamento
for site in launch_sites:
    folium.Marker(
        location=[site['latitude'], site['longitude']],
        popup=site['name'],
        icon=folium.Icon(color='blue', icon='info-sign')
    ).add_to(site_map)

# Exibir o mapa
site_map



First, let's try to add each site's location on a map using site's latitude and longitude coordinates


The following dataset with the name `spacex_launch_geo.csv` is an augmented dataset with latitude and longitude added for each site.


In [8]:
# Download and read the `spacex_launch_geo.csv`
from js import fetch
import io

URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
resp = await fetch(URL)
spacex_csv_file = io.BytesIO((await resp.arrayBuffer()).to_py())
spacex_df=pd.read_csv(spacex_csv_file)

Now, you can take a look at what are the coordinates for each site.


In [9]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


Above coordinates are just plain numbers that can not give you any intuitive insights about where are those launch sites. If you are very good at geography, you can interpret those numbers directly in your mind. If not, that's fine too. Let's visualize those locations by pinning them on a map.


We first need to create a folium `Map` object, with an initial center location to be NASA Johnson Space Center at Houston, Texas.


In [10]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

We could use `folium.Circle` to add a highlighted circle area with a text label on a specific coordinate. For example,


In [11]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

and you should find a small yellow circle near the city of Houston and you can zoom-in to see a larger circle.


Now, let's add a circle for each launch site in data frame `launch_sites`


*TODO:*  Create and add `folium.Circle` and `folium.Marker` for each launch site on the site map


An example of folium.Circle:


`folium.Circle(coordinate, radius=1000, color='#000000', fill=True).add_child(folium.Popup(...))`


An example of folium.Marker:


`folium.map.Marker(coordinate, icon=DivIcon(icon_size=(20,20),icon_anchor=(0,0), html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'label', ))`


In [12]:
# Initial the map
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)
# Download and read the `spacex_launch_geo.csv`
URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
resp = await fetch(URL)
spacex_csv_file = io.BytesIO((await resp.arrayBuffer()).to_py())
spacex_df = pd.read_csv(spacex_csv_file)

# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]

# Start location: NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]

# Initialize a folium map
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

# Add a circle marker for NASA Johnson Space Center
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(
    folium.Popup('NASA Johnson Space Center'))
marker = folium.map.Marker(
    nasa_coordinate,
    icon=DivIcon(
        icon_size=(20, 20),
        icon_anchor=(0, 0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
    )
)
site_map.add_child(circle)
site_map.add_child(marker)

# Add launch site markers
for index, row in launch_sites_df.iterrows():
    launch_coordinate = [row['Lat'], row['Long']]
    # Add a circle at the launch site
    folium.Circle(
        location=launch_coordinate,
        radius=1000,
        color='blue',
        fill=True,
        fill_opacity=0.5
    ).add_child(folium.Popup(row['Launch Site'])).add_to(site_map)
    
    # Add a marker at the launch site
    folium.map.Marker(
        launch_coordinate,
        icon=DivIcon(
            icon_size=(20, 20),
            icon_anchor=(0, 0),
            html='<div style="font-size: 12; color:blue;"><b>%s</b></div>' % row['Launch Site'],
        )
    ).add_to(site_map)

# Display the map
site_map


The generated map with marked launch sites should look similar to the following:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_markers.png">
</center>


Now, you can explore the map by zoom-in/out the marked areas
, and try to answer the following questions:

*   Are all launch sites in proximity to the Equator line?
*   Are all launch sites in very close proximity to the coast?

Also please try to explain your findings.


In [13]:
# Assuming spacex_df is already loaded with the relevant columns ['Launch Site', 'Lat', 'Long', 'class']
# 'class' column contains 1 for success and 0 for failure

# Reinitialize the map centered at NASA JSC for visualization
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

# Iterate through each row in the spacex_df DataFrame and add a marker to the map
for index, row in spacex_df.iterrows():
    # Define the color based on the launch class
    marker_color = 'green' if row['class'] == 1 else 'red'
    
    # Add a marker for the launch site
    folium.Marker(
        location=[row['Lat'], row['Long']],
        popup=f"Launch Site: {row['Launch Site']}<br>Class: {'Success' if row['class'] == 1 else 'Failure'}",
        icon=folium.Icon(color=marker_color, icon='info-sign')
    ).add_to(site_map)

# Display the map
site_map



Next, let's try to enhance the map by adding the launch outcomes for each site, and see which sites have high success rates.
Recall that data frame spacex_df has detailed launch records, and the `class` column indicates if this launch was successful or not


In [14]:
spacex_df.tail(10)

,Flight Number,Date,Time (UTC),Booster Version,Launch Site,Payload,Payload Mass (kg),Orbit,Customer,Landing Outcome,class,Lat,Long
46,43,2017-10-11,22:53:00,F9 FT B1031.2,KSC LC-39A,SES-11 / EchoStar 105,5200.00,GTO,SES EchoStar,Success (drone ship),1,28.573255,-80.646895
47,44,2017-10-30,19:34:00,F9 B4 B1042.1,KSC LC-39A,Koreasat 5A,3500.00,GTO,KT Corporation,Success (drone ship),1,28.573255,-80.646895
48,54,2018-05-11,20:14:00,F9 B5 B1046.1,KSC LC-39A,Bangabandhu-1,3600.00,GTO,Thales-Alenia/BTRC,Success (drone ship),1,28.573255,-80.646895
49,45,2017-12-15,15:36:00,F9 FT B1035.2,CCAFS SLC-40,SpaceX CRS-13,2205.00,LEO (ISS),NASA (CRS),Success (ground pad),1,28.563197,-80.576820
50,47,2018-01-08,1:00:00,F9 B4 B1043.1,CCAFS SLC-40,Zuma,3696.65,LEO,Northrop Grumman,Success (ground pad),1,28.563197,-80.576820
51,48,2018-01-31,21:25:00,F9 FT B1032.2,CCAFS SLC-40,GovSat-1 / SES-16,4230.00,GTO,SES,Controlled (ocean),0,28.563197,-80.576820
52,50,2018-03-06,5:33:00,F9 B4 B1044,CCAFS SLC-40,Hispasat 30W-6 PODSat,6092.00,GTO,Hispasat NovaWurks,No attempt,0,28.563197,-80.576820
53,52,2018-04-02,20:30:00,F9 B4 B1039.2,CCAFS SLC-40,SpaceX CRS-14,2647.00,LEO (ISS),NASA (CRS),No attempt,0,28.563197,-80.576820
54,53,2018-04-18,22:51:00,F9 B4 B1045.1,CCAFS SLC-40,Transiting Exoplanet Survey Satellite (TESS),362.00,HEO,NASA (LSP),Success (drone ship),1,28.563197,-80.576820
55,56,2018-06-04,4:45:00,F9 B4 B1040.2,CCAFS SLC-40,SES-12,5384.00,GTO,SES,No attempt,0,28.563197,-80.576820


Next, let's create markers for all launch records.
If a launch was successful `(class=1)`, then we use a green marker and if a launch was failed, we use a red marker `(class=0)`


Note that a launch only happens in one of the four launch sites, which means many launch records will have the exact same coordinate. Marker clusters can be a good way to simplify a map containing many markers having the same coordinate.


Let's first create a `MarkerCluster` object


In [15]:
marker_cluster = MarkerCluster()


*TODO:* Create a new column in `spacex_df` dataframe called `marker_color` to store the marker colors based on the `class` value


In [16]:
from folium.plugins import MarkerCluster

# Create a map centered at NASA JSC
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

# Initialize a MarkerCluster object
marker_cluster = MarkerCluster().add_to(site_map)

# Iterate through spacex_df to add markers for all launches
for index, row in spacex_df.iterrows():
    # Determine marker color based on the 'class' column
    marker_color = 'green' if row['class'] == 1 else 'red'
    
    # Add a marker for the launch outcome to the MarkerCluster
    folium.Marker(
        location=[row['Lat'], row['Long']],
        popup=f"Launch Site: {row['Launch Site']}<br>Outcome: {'Success' if row['class'] == 1 else 'Failure'}",
        icon=folium.Icon(color=marker_color)
    ).add_to(marker_cluster)

# Display the map
site_map


*TODO:* For each launch result in `spacex_df` data frame, add a `folium.Marker` to `marker_cluster`


In [18]:
from folium.plugins import MarkerCluster

# Initialize the map
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

# Create a MarkerCluster object
marker_cluster = MarkerCluster().add_to(site_map)

# Add markers to the cluster
for index, record in spacex_df.iterrows():
    # Determine marker color based on the 'class' column
    marker_color = 'green' if record['class'] == 1 else 'red'
    
    # Create a marker with coordinates, a popup, and the color
    marker = folium.Marker(
        location=[record['Lat'], record['Long']],
        popup=f"Launch Site: {record['Launch Site']}<br>Outcome: {'Success' if record['class'] == 1 else 'Failure'}",
        icon=folium.Icon(color=marker_color)
    )
    
    # Add the marker to the cluster
    marker.add_to(marker_cluster)

# Add the marker cluster to the map
site_map.add_child(marker_cluster)

# Display the map
site_map


Your updated map may look like the following screenshots:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster.png">
</center>


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster_zoomed.png">
</center>


From the color-labeled markers in marker clusters, you should be able to easily identify which launch sites have relatively high success rates.


In [20]:
# TASK 3: Calculate the distances between a launch site to its proximities
from math import radians, sin, cos, sqrt, atan2

# Define a function to calculate the Haversine distance
def haversine(lat1, lon1, lat2, lon2):
    # Convert latitude and longitude from degrees to radians
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    
    # Haversine formula
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    r = 6371  # Radius of the Earth in kilometers
    return r * c

# Example: Calculate distance from a launch site to a point of interest
launch_site_lat, launch_site_lon = 28.562302, -80.577356  # Example coordinates for CCAFS LC-40
poi_lat, poi_lon = 28.563197, -80.576820  # Example point of interest near CCAFS LC-40

distance = haversine(launch_site_lat, launch_site_lon, poi_lat, poi_lon)
print(f"Distance from launch site to point of interest: {distance:.2f} km")

# Example: Add a line between launch site and point of interest
folium.PolyLine(
    locations=[[launch_site_lat, launch_site_lon], [poi_lat, poi_lon]],
    color='blue',
    weight=2,
    tooltip=f"Distance: {distance:.2f} km"
).add_to(site_map)

# Display updated map
site_map



Distance from launch site to point of interest: 0.11 km


Next, we need to explore and analyze the proximities of launch sites.


Let's first add a `MousePosition` on the map to get coordinate for a mouse over a point on the map. As such, while you are exploring the map, you can easily find the coordinates of any points of interests (such as railway)


In [21]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


In [22]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

*TODO:* Mark down a point on the closest coastline using MousePosition and calculate the distance between the coastline point and the launch site.


In [23]:
from folium.plugins import MousePosition

# Add Mouse Position to the map to get coordinates interactively
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position="topright",
    separator=" Long: ",
    empty_string="NaN",
    lng_first=False,
    num_digits=5,
    prefix="Lat:",
    lat_formatter=formatter,
    lng_formatter=formatter,
)

# Add the MousePosition plugin to the site map
site_map.add_child(mouse_position)
site_map


In [35]:
from folium.plugins import MarkerCluster

# Create a map centered at NASA JSC
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

# Initialize a MarkerCluster object
marker_cluster = MarkerCluster().add_to(site_map)

# Iterate through spacex_df to add markers for all launches
for index, row in spacex_df.iterrows():
    # Determine marker color based on the 'class' column
    marker_color = 'green' if row['class'] == 1 else 'red'
    
    # Add a marker for the launch outcome to the MarkerCluster
    folium.Marker(
        location=[row['Lat'], row['Long']],
        popup=f"Launch Site: {row['Launch Site']}<br>Outcome: {'Success' if row['class'] == 1 else 'Failure'}",
        icon=folium.Icon(color=marker_color)
    ).add_to(marker_cluster)

# Display the map
site_map


*TODO:* Draw a `PolyLine` between a launch site to the selected coastline point


In [36]:
from folium.plugins import MarkerCluster, MousePosition
from folium.features import DivIcon
from math import sin, cos, sqrt, atan2, radians

# Função para calcular distância
def calculate_distance(lat1, lon1, lat2, lon2):
    R = 6373.0  # Raio aproximado da Terra em km
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    return R * c

# Coordenadas do NASA JSC
nasa_coordinate = [29.559684888503615, -95.0830971930759]

# Criar um mapa centrado no NASA JSC
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

# Adicionar marcador para o NASA JSC
nasa_circle = folium.Circle(
    nasa_coordinate, radius=1000, color='#d35400', fill=True
).add_child(folium.Popup('NASA Johnson Space Center'))
nasa_marker = folium.Marker(
    nasa_coordinate,
    icon=DivIcon(
        icon_size=(20, 20),
        icon_anchor=(0, 0),
        html='<div style="font-size: 12; color:#d35400;"><b>NASA JSC</b></div>',
    ),
)
site_map.add_child(nasa_circle)
site_map.add_child(nasa_marker)

# Inicializar um objeto MarkerCluster
marker_cluster = MarkerCluster().add_to(site_map)

# Coordenadas adicionais para cidade, ferrovia e rodovia
additional_points = [
    {"name": "Highway", "lat": 28.5634, "lon": -80.5768, "type": "highway"},
    {"name": "Railway", "lat": 28.5627, "lon": -80.5770, "type": "railway"},
    {"name": "City", "lat": 28.6082, "lon": -80.8076, "type": "city"}
]

# Coordenadas dos locais de lançamento e ponto da costa
launch_sites = [
    {"name": "CCAFS LC-40", "lat": 28.562302, "lon": -80.577356, "coast_lat": 28.56277, "coast_lon": -80.57163},
    {"name": "CCAFS SLC-40", "lat": 28.563197, "lon": -80.576820, "coast_lat": 28.56278, "coast_lon": -80.57150},
    {"name": "KSC LC-39A", "lat": 28.573255, "lon": -80.646895, "coast_lat": 28.5721, "coast_lon": -80.6468},
    {"name": "VAFB SLC-4E", "lat": 34.632834, "lon": -120.610745, "coast_lat": 34.63305, "coast_lon": -120.61058},
]

# Adicionar marcadores para todos os lançamentos no DataFrame
for index, row in spacex_df.iterrows():
    marker_color = 'green' if row['class'] == 1 else 'red'
    folium.Marker(
        location=[row['Lat'], row['Long']],
        popup=f"Launch Site: {row['Launch Site']}<br>Outcome: {'Success' if row['class'] == 1 else 'Failure'}",
        icon=folium.Icon(color=marker_color)
    ).add_to(marker_cluster)

# Adicionar marcadores e linhas para locais de lançamento e pontos próximos
for site in launch_sites:
    launch_lat, launch_lon = site["lat"], site["lon"]
    coast_lat, coast_lon = site["coast_lat"], site["coast_lon"]
    distance = calculate_distance(launch_lat, launch_lon, coast_lat, coast_lon)

    # Adicionar marcador no local de lançamento
    folium.Marker(
        [launch_lat, launch_lon],
        popup=f"{site['name']}",
        icon=folium.Icon(color='blue', icon='info-sign')
    ).add_to(site_map)

    # Adicionar marcador na costa
    folium.Marker(
        [coast_lat, coast_lon],
        icon=DivIcon(
            icon_size=(20, 20),
            icon_anchor=(0, 0),
            html=f'<div style="font-size: 12; color:#d35400;"><b>{distance:.2f} KM</b></div>',
        ),
    ).add_to(site_map)

    # Adicionar linha entre local de lançamento e costa
    folium.PolyLine(
        locations=[[launch_lat, launch_lon], [coast_lat, coast_lon]],
        weight=2,
        color='blue'
    ).add_to(site_map)

# Adicionar marcadores para pontos adicionais
for point in additional_points:
    icon_color = "green" if point["type"] == "highway" else "red" if point["type"] == "railway" else "orange"
    folium.Marker(
        [point["lat"], point["lon"]],
        popup=point["name"],
        icon=folium.Icon(color=icon_color, icon='info-sign')
    ).add_to(site_map)

# Adicionar MousePosition para capturar coordenadas
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)
site_map.add_child(mouse_position)

# Exibir o mapa final
site_map


Your updated map with distance line should look like the following screenshot:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_distance.png">
</center>


*TODO:* Similarly, you can draw a line betwee a launch site to its closest city, railway, highway, etc. You need to use `MousePosition` to find the their coordinates on the map first


A railway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/railway.png">
</center>


A highway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/highway.png">
</center>


A city map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/city.png">
</center>


In [ ]:
# Create a marker with distance to a closest city, railway, highway, etc.
# Draw a line between the marker to the launch site


In [37]:
# Define the closest feature coordinates (e.g., city, railway, highway)
# Example coordinates for a city near a launch site (use MousePosition to identify these)
city_coords = [28.4011, -80.6012]  # Example for closest city
railway_coords = [28.5721, -80.5853]  # Example for closest railway
highway_coords = [28.5628, -80.5707]  # Example for closest highway

# Launch site coordinates (example: CCAFS LC-40)
launch_site_coords = [28.562302, -80.577356]

# Calculate distances to the features
distance_to_city = calculate_distance(
    launch_site_coords[0], launch_site_coords[1],
    city_coords[0], city_coords[1]
)
distance_to_railway = calculate_distance(
    launch_site_coords[0], launch_site_coords[1],
    railway_coords[0], railway_coords[1]
)
distance_to_highway = calculate_distance(
    launch_site_coords[0], launch_site_coords[1],
    highway_coords[0], highway_coords[1]
)

# Add markers for the features
city_marker = folium.Marker(
    location=city_coords,
    popup="Closest City",
    icon=folium.Icon(color='blue', icon='home')
)
railway_marker = folium.Marker(
    location=railway_coords,
    popup="Closest Railway",
    icon=folium.Icon(color='purple', icon='train')
)
highway_marker = folium.Marker(
    location=highway_coords,
    popup="Closest Highway",
    icon=folium.Icon(color='green', icon='road')
)

# Add these markers to the map
site_map.add_child(city_marker)
site_map.add_child(railway_marker)
site_map.add_child(highway_marker)

# Draw lines between the launch site and the features
line_to_city = folium.PolyLine(
    locations=[launch_site_coords, city_coords],
    color='blue',
    weight=2
)
line_to_railway = folium.PolyLine(
    locations=[launch_site_coords, railway_coords],
    color='purple',
    weight=2
)
line_to_highway = folium.PolyLine(
    locations=[launch_site_coords, highway_coords],
    color='green',
    weight=2
)

# Add the lines to the map
site_map.add_child(line_to_city)
site_map.add_child(line_to_railway)
site_map.add_child(line_to_highway)

# Add markers with distances to each feature
city_distance_marker = folium.Marker(
    location=[
        (launch_site_coords[0] + city_coords[0]) / 2,  # Midpoint latitude
        (launch_site_coords[1] + city_coords[1]) / 2   # Midpoint longitude
    ],
    icon=DivIcon(
        icon_size=(150, 36),
        icon_anchor=(0, 0),
        html='<div style="font-size: 12; color: blue;"><b>{:.2f} km</b></div>'.format(distance_to_city)
    )
)
railway_distance_marker = folium.Marker(
    location=[
        (launch_site_coords[0] + railway_coords[0]) / 2,  # Midpoint latitude
        (launch_site_coords[1] + railway_coords[1]) / 2   # Midpoint longitude
    ],
    icon=DivIcon(
        icon_size=(150, 36),
        icon_anchor=(0, 0),
        html='<div style="font-size: 12; color: purple;"><b>{:.2f} km</b></div>'.format(distance_to_railway)
    )
)
highway_distance_marker = folium.Marker(
    location=[
        (launch_site_coords[0] + highway_coords[0]) / 2,  # Midpoint latitude
        (launch_site_coords[1] + highway_coords[1]) / 2   # Midpoint longitude
    ],
    icon=DivIcon(
        icon_size=(150, 36),
        icon_anchor=(0, 0),
        html='<div style="font-size: 12; color: green;"><b>{:.2f} km</b></div>'.format(distance_to_highway)
    )
)

# Add distance markers to the map
site_map.add_child(city_distance_marker)
site_map.add_child(railway_distance_marker)
site_map.add_child(highway_distance_marker)

# Display the updated map
site_map


In [39]:
from folium.plugins import MarkerCluster, MousePosition
from folium import DivIcon

# Initialize a map
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

# Add MousePosition plugin
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)
site_map.add_child(mouse_position)

# Initialize a MarkerCluster object for launches
marker_cluster = MarkerCluster().add_to(site_map)

# Add launch outcomes for all sites
for index, row in spacex_df.iterrows():
    marker_color = 'green' if row['class'] == 1 else 'red'
    folium.Marker(
        location=[row['Lat'], row['Long']],
        popup=f"Launch Site: {row['Launch Site']}<br>Outcome: {'Success' if row['class'] == 1 else 'Failure'}",
        icon=folium.Icon(color=marker_color)
    ).add_to(marker_cluster)

# Define proximity features for ALL launch sites
proximity_features = {
    "CCAFS LC-40": {
        "railway": {"coords": [28.5721, -80.5853], "name": "Closest Railway"},
        "highway": {"coords": [28.5628, -80.5707], "name": "Closest Highway"},
        "coastline": {"coords": [28.56367, -80.57163], "name": "Closest Coastline"},
        "city": {"coords": [28.4011, -80.6012], "name": "Closest City"},
    },
    "VAFB SLC-4E": {
        "railway": {"coords": [34.6351, -120.6108], "name": "Closest Railway"},
        "highway": {"coords": [34.6325, -120.6090], "name": "Closest Highway"},
        "coastline": {"coords": [34.632834, -120.6057], "name": "Closest Coastline"},
        "city": {"coords": [34.6383, -120.4579], "name": "Closest City"},
    },
    "KSC LC-39A": {
        "railway": {"coords": [28.5720, -80.6550], "name": "Closest Railway"},
        "highway": {"coords": [28.5732, -80.6492], "name": "Closest Highway"},
        "coastline": {"coords": [28.573255, -80.6460], "name": "Closest Coastline"},
        "city": {"coords": [28.5383, -81.3792], "name": "Closest City"},
    },
}

# Loop through each launch site
for site, features in proximity_features.items():
    # Get launch site coordinates
    launch_site_coords = launch_sites_df[launch_sites_df['Launch Site'] == site][['Lat', 'Long']].values[0]
    
    # Add feature markers and lines
    for feature, details in features.items():
        # Calculate distance
        distance = calculate_distance(
            launch_site_coords[0], launch_site_coords[1],
            details["coords"][0], details["coords"][1]
        )
        
        # Add marker for the feature
        folium.Marker(
            location=details["coords"],
            popup=f"{details['name']}: {distance:.2f} km",
            icon=folium.Icon(color='blue', icon='info-sign')
        ).add_to(site_map)

        # Draw line between launch site and feature
        folium.PolyLine(
            locations=[launch_site_coords, details["coords"]],
            color='blue',
            weight=2,
        ).add_to(site_map)

        # Add distance label
        mid_lat = (launch_site_coords[0] + details["coords"][0]) / 2
        mid_lon = (launch_site_coords[1] + details["coords"][1]) / 2
        folium.Marker(
            location=[mid_lat, mid_lon],
            icon=DivIcon(
                icon_size=(150, 36),
                icon_anchor=(0, 0),
                html=f'<div style="font-size: 12; color:blue;"><b>{distance:.2f} km</b></div>',
            ),
        ).add_to(site_map)

# Display the updated map
site_map


After you plot distance lines to the proximities, you can answer the following questions easily:

*   Are launch sites in close proximity to railways?
*   Are launch sites in close proximity to highways?
*   Are launch sites in close proximity to coastline?
*   Do launch sites keep certain distance away from cities?

Also please try to explain your findings.


# Next Steps:

Now you have discovered many interesting insights related to the launch sites' location using folium, in a very interactive way. Next, you will need to build a dashboard using Ploty Dash on detailed launch records.


## Authors


[Pratiksha Verma](https://www.linkedin.com/in/pratiksha-verma-6487561b1/)


<!--## Change Log--!>


<!--| Date (YYYY-MM-DD) | Version | Changed By      | Change Description      |
| ----------------- | ------- | -------------   | ----------------------- |
| 2022-11-09        | 1.0     | Pratiksha Verma | Converted initial version to Jupyterlite|--!>


### <h3 align="center"> IBM Corporation 2022. All rights reserved. <h3/>
